In [2]:
import os
import pandas as pd
from pulp import LpVariable, LpMinimize, LpProblem, LpStatus, lpSum

# Load the data from CSV file
file_path = os.path.join("fau_clinic_shifts.csv")
fau_clinic_df = pd.read_csv(file_path, index_col=0)
fau_clinic_df = fau_clinic_df.rename(index={' 6:00-07:00': '06:00 – 07:00', '12:00-13:00': '12:00 – 13:00'})
fau_clinic_df = fau_clinic_df.fillna(0).applymap(lambda x: 1 if x == "X" else x)

# Extract relevant data for optimization
shifts = fau_clinic_df.drop(index=["Wage rate per 8h shift (EUR)"], columns=["Avg_Patient_Number"]).values
shift_num = shifts.shape[1]
time_windows = shifts.shape[0]
avg_patient_number = fau_clinic_df["Avg_Patient_Number"].values
wages_per_shift = fau_clinic_df.loc["Wage rate per 8h shift (EUR)", :].values.astype(int)
service_level = 0.25

# Create PuLP problem and variables
lp_prob = LpProblem("scheduling_nurses", LpMinimize)
num_nurses = [LpVariable(f"num_nurses_{i}", lowBound=0, cat="Integer") for i in range(shift_num)]

# Objective function: Minimize the total cost (wages) for nurses
lp_prob += lpSum(wages_per_shift[shift_ind] * num_nurses[shift_ind] for shift_ind in range(shift_num))

# Constraints: Ensure service level is met for each time window
for t in range(time_windows):
    lp_prob += lpSum(shifts[t, i] * num_nurses[i] for i in range(shift_num)) >= (avg_patient_number[t] * service_level)

# Solve the problem
lp_prob.solve()

# Print the results
print("Status:", LpStatus[lp_prob.status], "\n")

for shift_ind, var in enumerate(num_nurses):
    print(f"The number of nurses needed for shift {shift_ind} is {int(var.value())} nurses")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/rehnuma/opt/miniconda3/envs/pa/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/gp/qj9g7b5j23bd8lvcshwyhs0w0000gn/T/4fc1d22643da4c98a0d4df8963049ee8-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/gp/qj9g7b5j23bd8lvcshwyhs0w0000gn/T/4fc1d22643da4c98a0d4df8963049ee8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 29 COLUMNS
At line 63 RHS
At line 88 BOUNDS
At line 92 ENDATA
Problem MODEL has 24 rows, 3 columns and 24 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1126.75 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 1166 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of whi